In [1]:
from tensorflow.keras.layers import BatchNormalization
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

In [2]:
NAME = f'wri-cnn-64x4-{int(time.time())}'
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

In [3]:
def pickle_load(x_name, y_name):    
    pickle_in = open(f'{x_name}.pickle', 'rb')
    X = pickle.load(pickle_in)
    pickle_in.close()

    pickle_in = open(f'{y_name}.pickle', 'rb')
    y = pickle.load(pickle_in)
    pickle_in.close()
    return X, y

def getConvModel(input_shape, num_kernels, hidden_neurons, num_layers):
    model = Sequential()
    model.add(Conv2D(num_kernels, (3, 3), input_shape=input_shape, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    for i in range(0, num_layers - 2):
        model.add(Conv2D(num_kernels, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(hidden_neurons, activation='relu'))
    model.add(Dropout(0.5))
#     model.add(Dense(hidden_neurons, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    return model

In [4]:
X_train, y_train = pickle_load('X_train', 'y_train')
X_val, y_val = pickle_load('X_val', 'y_val')

In [ ]:
conv_model = getConvModel(X_train.shape[1:], 64, 64, 4)
conv_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=10, callbacks=[tensorboard])

In [ ]:
# serialize model to JSON
model_json = conv_model.to_json()
with open("conv_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
conv_model.save_weights("conv_model.h5")
print("Saved model to disk")